In [126]:
import os
import requests

# Get PDF document path
pdf_path = "/kaggle/input/rag-pdf/rag_pdf.pdf"


In [130]:
!pip install PyMuPDF
!pip install tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [131]:
import fitz 
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """Opens a PDF file, reads its text content page by page, and collects statistics."""
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({"page_number": page_number,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(", ")),
                                "page_token_count": len(text) / 4,
                                "text": text})
    return pages_and_texts
pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': 0,
  'page_char_count': 34,
  'page_word_count': 6,
  'page_sentence_count_raw': 1,
  'page_token_count': 8.5,
  'text': 'Your car insurance  policy booklet'},
 {'page_number': 1,
  'page_char_count': 1799,
  'page_word_count': 354,
  'page_sentence_count_raw': 4,
  'page_token_count': 449.75,
  'text': 'Page 2 FAQs\t 3 Glossary \t 4 Making a claim\t 6 What your cover includes\t 8 Section 1: Liability \t 11 Section 2: Fire and theft \t 14 Section 3: Courtesy car \t 17 Section 4: Accidental damage \t 18 Section 5: Windscreen damage \t 20 Section 6: Personal benefits \t 21 Section 7: Motor Legal Cover \t 23 Section 8: Guaranteed Hire Car Plus \t 28 Section 9: Protected No Claim Discount\t 30 Where you can drive\t 31 Losses we don’t cover \t 33 Other conditions you need to know about\t 36 How the policy works \t 37 Everything else \t 41 If you have a complaint \t 42 If you’re in an accident\t 43 How to get in touch \t Back cover Contents Welcome to Churchill This booklet 

In [132]:
import random

random.sample(pages_and_texts, k=2)

[{'page_number': 38,
  'page_char_count': 3717,
  'page_word_count': 729,
  'page_sentence_count_raw': 24,
  'page_token_count': 929.25,
  'text': 'If you’ve made a claim, and/or used your Green  Flag cover, before the cancellation or removal  happens For the policy and car insurance cover options,  e.g. Protected No Claim Discount: > We will not refund any car insurance  premium if you have made a car insurance  claim or if one has been made against you  during the period of cover (regardless of  whether you pay annually or by monthly  instalments under a credit agreement). > If you pay by instalments under a credit  agreement, you must pay to us (1) all  instalment payments that have already  fallen due under the credit agreement and  remain unpaid, and (2) the total remaining  balance under the credit agreement. > If we agree to pay your claim and you have  not paid the amounts due to us under (1)  and (2) above, we may reduce the amount  that we pay in settlement of your claim by  

EDA




In [133]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,0,34,6,1,8.50,Your car insurance policy booklet
1,1,1799,354,4,449.75,Page 2 FAQs\t 3 Glossary \t 4 Making a claim\t...
2,2,3720,716,20,930.00,Page 3 FAQs How much will you pay if my car is...
3,3,3062,548,17,765.50,Page 4 About the glossary When we use these wo...
4,4,2809,530,24,702.25,Glossary Loss of any limb A limb severed at or...


In [134]:
df.tail()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
39,39,2489,534,18,622.25,"premium, unless there has been any fraudulent ..."
40,40,3819,688,25,954.75,Page 41 Everything else Everything else Our co...
41,41,2583,493,14,645.75,Page 42 If you have a complaint How we can hel...
42,42,2383,445,18,595.75,Independent review If we don’t complete our in...
43,43,864,141,8,216.00,Need to claim? 0345 878 6261 Windscreen claims...


In [135]:
df.shape

(44, 6)

In [136]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,44.00,44.00,44.00,44.00,44.00
mean,21.50,2428.98,464.68,15.73,607.24
std,12.85,881.95,171.13,8.76,220.49
min,0.00,34.00,6.00,1.00,8.50
25%,10.75,1834.25,353.50,9.00,458.56
50%,21.50,2544.00,496.50,14.50,636.00
75%,32.25,2925.75,553.75,20.25,731.44
max,43.00,3946.00,767.00,44.00,986.50


### Splitting pages into sentences

`Text -> split it into groups/chunks -> embed the groups/chunks -> use the embeddings`

In [137]:
from spacy.lang.en import English

nlp = English()

nlp.add_pipe("sentencizer")

for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/44 [00:00<?, ?it/s]

In [138]:
random.sample(pages_and_texts, k=1)

[{'page_number': 33,
  'page_char_count': 2402,
  'page_word_count': 437,
  'page_sentence_count_raw': 34,
  'page_token_count': 600.5,
  'text': 'Page 34 If your car is removed or seized by an authority \t ✘We won’t cover any loss or damage caused  by a government agency or other authority  that legally takes, keeps or destroys your car. \t ✘\t If you’re driving or using another car and it’s  taken by, or on behalf of, any government or  other authority, we won’t provide cover to get  it released. Contracts \t ✘We won’t cover any liability under any  separate agreement or contract you’ve  made, unless you’d have been responsible  even if that agreement or contract didn’t  exist. Radioactivity \t ✘We won’t cover any loss or damage to  property, any direct or indirect loss, or any  expense, or any liability caused, or  contributed to, by: •\t Ionising radiation or radioactive  contamination from nuclear fuel or waste. •\t Radioactive, toxic, explosive or other  dangerous properties of n

In [139]:
df = pd.DataFrame(pages_and_texts)
df

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text,sentences,page_sentence_count_spacy
0,0,34,6,1,8.50,Your car insurance policy booklet,[Your car insurance policy booklet],1
1,1,1799,354,4,449.75,Page 2 FAQs\t 3 Glossary \t 4 Making a claim\t...,[Page 2 FAQs\t 3 Glossary \t 4 Making a claim\...,12
2,2,3720,716,20,930.00,Page 3 FAQs How much will you pay if my car is...,[Page 3 FAQs How much will you pay if my car i...,51
3,3,3062,548,17,765.50,Page 4 About the glossary When we use these wo...,[Page 4 About the glossary When we use these w...,28
4,4,2809,530,24,702.25,Glossary Loss of any limb A limb severed at or...,[Glossary Loss of any limb A limb severed at o...,37
5,5,2725,537,21,681.25,Page 6 Making a claim If you need to claim The...,[Page 6 Making a claim If you need to claim Th...,26
6,6,3946,767,27,986.50,Page 7 Making a claim How repairs and replacem...,[Page 7 Making a claim How repairs and replace...,33
7,7,1402,261,4,350.50,Page 8 We’ll provide cover up to the amounts s...,[Page 8 We’ll provide cover up to the amounts ...,4
8,8,1047,194,2,261.75,"Page 9 What your cover includes Third Party, ...","[Page 9 What your cover includes Third Party, ...",1
9,9,1132,210,5,283.00,"Third Party, Fire and Theft Essentials Compre...","[Third Party, Fire and Theft Essentials Compr...",1


In [140]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,44.00,44.00,44.00,44.00,44.00,44.00
mean,21.50,2428.98,464.68,15.73,607.24,20.95
std,12.85,881.95,171.13,8.76,220.49,10.46
min,0.00,34.00,6.00,1.00,8.50,1.00
25%,10.75,1834.25,353.50,9.00,458.56,14.75
50%,21.50,2544.00,496.50,14.50,636.00,23.50
75%,32.25,2925.75,553.75,20.25,731.44,28.00
max,43.00,3946.00,767.00,44.00,986.50,51.00


In [141]:
chunk_size = 15
def split_list(input_list: list[str], 
               slice_size: int=chunk_size) -> list[list[str]]:
    return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]

for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/44 [00:00<?, ?it/s]

In [142]:
random.sample(pages_and_texts,k=1)

[{'page_number': 40,
  'page_char_count': 3819,
  'page_word_count': 688,
  'page_sentence_count_raw': 25,
  'page_token_count': 954.75,
  'text': 'Page 41 Everything else Everything else Our contract with you We aim to always be fair and reasonable and to  act quickly whenever you need to make a claim  under this policy. If you feel we haven’t met this,  we’ll try to do everything possible to deal with  your complaint quickly and fairly. This policy is evidence of the contract between  you and us, U K Insurance Limited, and is based  on information you’ve given to us. In return for receiving and accepting the  premium, we’ll provide insurance under this  policy for the sections shown on your car  insurance details.  The laws that apply to this contract You and we  may choose which law will apply to this policy.  Unless both parties agree otherwise, English law  will apply. However, if you are resident in Jersey,  Guernsey, Alderney or the Isle of Man, the law  of the island where you 

In [143]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,44.00,44.00,44.00,44.00,44.00,44.00,44.00
mean,21.50,2428.98,464.68,15.73,607.24,20.95,1.82
std,12.85,881.95,171.13,8.76,220.49,10.46,0.69
min,0.00,34.00,6.00,1.00,8.50,1.00,1.00
25%,10.75,1834.25,353.50,9.00,458.56,14.75,1.00
50%,21.50,2544.00,496.50,14.50,636.00,23.50,2.00
75%,32.25,2925.75,553.75,20.25,731.44,28.00,2.00
max,43.00,3946.00,767.00,44.00,986.50,51.00,4.00


### Splitting each chunks



In [144]:
import re
from tqdm import tqdm
def split_text_into_sentences(text):
    # Regular expression to handle bullet points and symbols, ensuring proper sentence splitting
    bullet_points_and_symbols = re.compile(r'(\s*[•>\-\*✘]\s.*(?:\n|$))')
    sentence_endings = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s')
    
    text_parts = bullet_points_and_symbols.split(text)
    sentences = []
    for part in text_parts:
        if bullet_points_and_symbols.match(part):
            sentences.append(part.strip())
        else:
            sentences.extend(sentence_endings.split(part))
    
    unwanted_symbols = re.compile(r'^\s*[•>\-\*✘]\s*.*')
    return [sentence.strip() for sentence in sentences if sentence.strip() and not unwanted_symbols.match(sentence)]

def chunk_sentences(sentences, chunk_size=20):
    return [sentences[i:i + chunk_size] for i in range(0, len(sentences), chunk_size)]

pages_and_chunks = []

for item in tqdm(pages_and_texts):
    page_number = item["page_number"]
    text = item["text"]

    sentences = split_text_into_sentences(text)

    sentence_chunks = chunk_sentences(sentences, chunk_size=10)  # You can adjust the chunk size here

    for sentence_chunk in sentence_chunks:
        chunk_dict = {}
        chunk_dict["page_number"] = page_number

        joined_sentence_chunk = " ".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)  # convert ".A" to ". A" (only for capital letter)
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len(joined_sentence_chunk.split())
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk.split())  # Assuming 1 token = 1 word for approximation

        pages_and_chunks.append(chunk_dict)

print(len(pages_and_chunks))


100%|██████████| 44/44 [00:00<00:00, 3002.32it/s]

48


In [145]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 43,
  'sentence_chunk': 'Need to claim? 0345 878 6261 Windscreen claims 0800 328 9150 If you have Essentials, Comprehensive or Comprehensive Plus cover Motor legal helpline 0345 246 2408 If you have Motor Legal Cover DriveSure help Email Support@churchill.com Call 0345 878 6375 If you have a question about telematics Help with anything else FAQs churchill.com/faqs Call 0345 877 6680 If you would like a Braille, large print or audio version of your documents, please let us know. Churchill insurance policies are underwritten by U K Insurance Limited, Registered office: The Wharf, Neville Street, Leeds, LS1 4AZ. Registered in England and Wales No.1179980. U K Insurance Limited is authorised by the Prudential Regulation Authority and regulated by the Financial Conduct Authority and the Prudential Regulation Authority. Calls may be recorded. B4C CH M PB 0923 How to get in touch',
  'chunk_char_count': 860,
  'chunk_word_count': 137,
  'chunk_token_count': 137}]

In [146]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,48.00,48.00,48.00,48.00
mean,21.56,595.75,104.52,104.52
std,13.45,455.34,79.13,79.13
min,0.00,33.00,5.00,5.00
25%,9.75,165.75,29.50,29.50
50%,21.50,567.50,102.00,102.00
75%,33.25,945.00,164.25,164.25
max,43.00,1595.00,265.00,265.00


In [147]:
 df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
0,0,Your car insurance policy booklet,33,5,5
1,1,Page 2 FAQs\t 3 Glossary \t 4 Making a claim\t...,733,131,131
2,2,Page 3 FAQs How much will you pay if my car is...,758,151,151
3,3,Page 4 About the glossary When we use these wo...,1392,232,232
4,3,"This identification may be by type, informatio...",146,23,23


In [148]:
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(3).iterrows():
    print(f'Chunk token count : {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count : 14 | Text: Page 25 Section 7: Motor Legal Cover You’re not covered for 	 ✘We don’t cover:
Chunk token count : 16 | Text: Settling or ending your claim Offers to settle your claim You must tell us if anyone:
Chunk token count : 21 | Text: You can find this on your certificate of motor insurance and car insurance details. Policy The policy is made up of:


In [149]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': 1,
  'sentence_chunk': 'Page 2 FAQs\t 3 Glossary \t 4 Making a claim\t 6 What your cover includes\t 8 Section 1: Liability \t 11 Section 2: Fire and theft \t 14 Section 3: Courtesy car \t 17 Section 4: Accidental damage \t 18 Section 5: Windscreen damage \t 20 Section 6: Personal benefits \t 21 Section 7: Motor Legal Cover \t 23 Section 8: Guaranteed Hire Car Plus \t 28 Section 9: Protected No Claim Discount\t 30 Where you can drive\t 31 Losses we don’t cover \t 33 Other conditions you need to know about\t 36 How the policy works \t 37 Everything else \t 41 If you have a complaint \t 42 If you’re in an accident\t 43 How to get in touch \t Back cover Contents Welcome to Churchill This booklet tells you about your car insurance About the policy The policy is made up of:',
  'chunk_char_count': 733,
  'chunk_word_count': 131,
  'chunk_token_count': 131},
 {'page_number': 2,
  'sentence_chunk': 'Page 3 FAQs How much will you pay if my car is damaged? Where damage to your c

In [150]:
random.sample(pages_and_chunks_over_min_token_len, k=1)

[{'page_number': 8,
  'sentence_chunk': 'Page 9 What your cover includes Third Party, Fire and Theft Essentials Comprehensive Comprehensive Plus Section 4: Accidental damage Accidental damage to your car 8 4 Market value Child car seat replacement 4 Unlimited In-car entertainment fitted in your car when it was made 4 Unlimited In-car entertainment fitted in your car after it was made 8 4 £1,000 4 £2,000 Removable electronic equipment 4 £1,000 4 £2,000 Misfuelling 4 Market value Lost or damaged car keys 4 £1,000 Section 5: Windscreen damage If you use an approved windscreen supplier 8 4 Market value If you choose a different supplier for repairs 4 The amount shown in your car insurance details If you choose a different supplier for replacement 4 The amount shown in your car insurance details Section 6: Personal benefits New car replacement 8 8 4 Market value Personal belongings 4 £250 4 £500 Personal accident 4 £5,000 4 £10,000 Medical expenses 4 £200 4 £400 Hotel expenses 8 4 £250 4 £ 

### Embedding our text chunks

In [151]:
!pip install sentence-transformers # for embedding models

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [152]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cuda")


for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])


  0%|          | 0/34 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  9%|▉         | 3/34 [00:00<00:01, 20.07it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 97%|█████████▋| 33/34 [00:01<00:00, 21.36it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 34/34 [00:01<00:00, 21.70it/s]


In [153]:
pages_and_chunks_over_min_token_len[0]["embedding"].shape

(768,)

In [154]:
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]
text_chunks[9]

'Section 1: Liability Liability to other people (third parties) We’ll cover you if you’re found to be legally responsible for an accident. If you’re on: Third Party, Fire and Theft Essentials Comprehensive Comprehensive Plus the cover in this section depends on the type of insurance you have. If you cause an accident Included with: TPFT Essential Comp Comp+ We’ll provide cover if you’re found to be legally responsible to other people for an accident involving your car that:'

In [155]:
len(text_chunks)

34

In [156]:
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=16, # Embed all texts in batches
                                               convert_to_tensor=True)
text_chunk_embeddings[0]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

tensor([-3.5948e-02,  3.9779e-02,  1.3696e-02,  2.3258e-02, -1.9896e-02,
         4.9702e-02,  4.9699e-02,  4.9442e-02,  2.3917e-02,  2.4623e-03,
         6.4488e-02,  3.7723e-03,  4.4912e-02, -1.9941e-02,  5.5085e-02,
        -3.9219e-02, -1.2904e-02, -1.6203e-02, -4.1437e-03, -1.2669e-03,
        -5.6369e-03,  2.4486e-02, -3.8288e-02,  3.2555e-02, -8.4123e-02,
         3.9614e-02,  1.2856e-02, -1.4015e-02,  5.0874e-02, -5.0425e-02,
         2.3520e-02,  3.9046e-02,  1.3039e-02, -6.8164e-02,  2.1225e-06,
         4.0265e-03,  2.0523e-02,  8.8283e-03,  1.5724e-02,  1.8648e-02,
         3.3418e-03,  6.1003e-02, -6.2450e-03,  1.4209e-02,  3.3478e-02,
         2.4154e-02,  1.5061e-02,  4.7106e-02,  1.2114e-02,  2.2850e-02,
        -2.1595e-02, -1.1684e-02, -5.6586e-02,  4.3610e-02,  9.4630e-02,
        -1.1101e-01,  5.2959e-03,  7.3410e-03,  3.7322e-02, -8.0984e-02,
        -2.4395e-02,  2.5159e-02,  2.4113e-02, -5.2333e-03, -2.1703e-02,
         4.9245e-02, -3.2247e-03, -4.2321e-02, -7.8

In [157]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
save_path = "embeddings.csv"
text_chunks_and_embeddings_df.to_csv(save_path, index=False)

In [158]:
text_chunks_and_embeddings_df_load = pd.read_csv(save_path)
text_chunks_and_embeddings_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1,Page 2 FAQs\t 3 Glossary \t 4 Making a claim\t...,733,131,131,[-3.59484553e-02 3.97789963e-02 1.36956088e-...
1,2,Page 3 FAQs How much will you pay if my car is...,758,151,151,[-4.14348766e-02 -9.52586383e-02 1.65855717e-...
2,3,Page 4 About the glossary When we use these wo...,1392,232,232,[-2.87577640e-02 -8.27380270e-02 1.67733629e-...
3,4,Glossary Loss of any limb A limb severed at or...,1059,186,186,[-2.03400366e-02 -4.59709615e-02 1.57789271e-...
4,5,Page 6 Making a claim If you need to claim The...,800,148,148,[-6.09188229e-02 6.13532308e-03 1.73641853e-...


### Semantic search
We can use the dot product or the cosine similarity depends on the size of the data we have.

In [159]:
import torch
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

text_chunks_and_embedding_df = pd.read_csv(save_path)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

embeddings = torch.tensor(np.stack(text_chunks_and_embedding_df["embedding"].tolist(), axis=0), dtype=torch.float32).to(device)
pages_and_chunks = text = text_chunks_and_embedding_df.to_dict(orient="records")

text_chunks_and_embeddings_df

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1,Page 2 FAQs\t 3 Glossary \t 4 Making a claim\t...,733,131,131,"[-0.035948455, 0.039778996, 0.013695609, 0.023..."
1,2,Page 3 FAQs How much will you pay if my car is...,758,151,151,"[-0.041434877, -0.09525864, 0.016585572, 0.025..."
2,3,Page 4 About the glossary When we use these wo...,1392,232,232,"[-0.028757764, -0.08273803, 0.0016773363, -0.0..."
3,4,Glossary Loss of any limb A limb severed at or...,1059,186,186,"[-0.020340037, -0.04597096, 0.015778927, 0.012..."
4,5,Page 6 Making a claim If you need to claim The...,800,148,148,"[-0.060918823, 0.006135323, 0.017364185, 0.030..."
5,6,Page 7 Making a claim How repairs and replacem...,188,36,36,"[-0.012906956, 0.041263822, 0.009727287, 0.043..."
6,7,Page 8 We’ll provide cover up to the amounts s...,1382,241,241,"[-0.047287278, -0.077034, 0.014966037, 0.03019..."
7,8,"Page 9 What your cover includes Third Party, F...",1029,176,176,"[-0.0700611, -0.046148766, 0.014801069, 0.0231..."
8,9,"Third Party, Fire and Theft Essentials Compreh...",1116,194,194,"[-0.05430105, -0.015806818, 0.01931512, 0.0321..."
9,10,Section 1: Liability Liability to other people...,477,78,78,"[-0.044253502, -0.055574816, -0.010450048, -0...."


In [160]:
embeddings.shape

torch.Size([34, 768])

In [161]:
from sentence_transformers import util

query = "Repairs"
print(f"Query : {query}")

query_embedding = embedding_model.encode(query, convert_to_tensor=True).to("cuda")

dot_scores = util.dot_score(query_embedding, embeddings)[0]

top_results = torch.topk(dot_scores, k=5)
top_results

Query : Repairs


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

torch.return_types.topk(
values=tensor([0.5657, 0.3826, 0.3685, 0.3284, 0.3127], device='cuda:0'),
indices=tensor([ 5, 17,  3, 14,  8], device='cuda:0'))

In [162]:
for score, idx in zip(top_results[0], top_results[1]):
    print(f"Score: {score:.4f}")
    print("Text")
    print(pages_and_chunks[idx]["sentence_chunk"])
    print("\n\n")


Score: 0.5657
Text
Page 7 Making a claim How repairs and replacements work This page explains how we’ll repair your car. We’ll only do this if it makes financial sense for us to do so. For more details, see:



Score: 0.3826
Text
Windscreen damage We’ll help put things right if the glass in your car is accidentally damaged. This cover is included with: Essentials Comprehensive Comprehensive Plus It’s not included with: Third Party, Fire and Theft What we’ll do We’ll pay to:



Score: 0.3685
Text
Glossary Loss of any limb A limb severed at or above the wrist or ankle, or the total and irrecoverable loss of use of a hand, arm, foot or leg. Main driver The person you declared was the main user of your car, and who’s shown as the main driver on your car insurance details. Market value The cost of replacing your car with another of the same make and model, and of a similar age, mileage, and condition at the time of the accident or loss. Modifications Any changes to your car’s standard speci

In [163]:
def retrieve_relevant_resources(query: str, n_resources_to_return: int=5):

    query_embedding = embedding_model.encode(query, convert_to_tensor=True).to("cuda")

    dot_scores = util.dot_score(query_embedding, embeddings)[0]

    scores, indices = torch.topk(dot_scores, k=n_resources_to_return)
    
    return scores, indices



In [164]:
retrieve_relevant_resources(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(tensor([0.5657, 0.3826, 0.3685, 0.3284, 0.3127], device='cuda:0'),
 tensor([ 5, 17,  3, 14,  8], device='cuda:0'))

In [165]:
def print_top_results_and_scores(query: str, n_resources_to_return: int=5):
    scores, indices = retrieve_relevant_resources(query, n_resources_to_return=n_resources_to_return)
    for score, idx in zip(scores, indices):
        print(f"Score: {score:.4f}")
        print("Text")
        print(pages_and_chunks[idx]["sentence_chunk"])
        print("\n\n")

In [166]:
print_top_results_and_scores(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Score: 0.5657
Text
Page 7 Making a claim How repairs and replacements work This page explains how we’ll repair your car. We’ll only do this if it makes financial sense for us to do so. For more details, see:



Score: 0.3826
Text
Windscreen damage We’ll help put things right if the glass in your car is accidentally damaged. This cover is included with: Essentials Comprehensive Comprehensive Plus It’s not included with: Third Party, Fire and Theft What we’ll do We’ll pay to:



Score: 0.3685
Text
Glossary Loss of any limb A limb severed at or above the wrist or ankle, or the total and irrecoverable loss of use of a hand, arm, foot or leg. Main driver The person you declared was the main user of your car, and who’s shown as the main driver on your car insurance details. Market value The cost of replacing your car with another of the same make and model, and of a similar age, mileage, and condition at the time of the accident or loss. Modifications Any changes to your car’s standard speci

Using Open source LLM's for Augmenting and getting the result


In [207]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [196]:
import kagglehub

# Download latest version
path = kagglehub.model_download("keras/gemma/keras/gemma_2b_en")

print("Path to model files:", path)

Attaching model 'keras/gemma/keras/gemma_2b_en' to your Kaggle notebook...


Path to model files: /kaggle/input/gemma/keras/gemma_2b_en/2


In [217]:
import keras
import keras_nlp
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("/kaggle/input/gemma/keras/gemma_2b_en/2") 

In [218]:
input_text = "What are Liabilities, and what role do they play?"

outputs = mistral_llm.generate(input_text, max_length=256)
print(outputs)

NameError: name 'mistral_llm' is not defined

In [211]:
query_list = [
    "How can I cancel my insurance policy or remove optional cover?",
    "Who can provide legal services for motor legal cover claims?"
]
query_list

['How can I cancel my insurance policy or remove optional cover?',
 'Who can provide legal services for motor legal cover claims?']

In [212]:
def prompt_formatter(query: str, 
                     context_items: list[dict]) -> str:
    
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

    base_prompt = """Based on the following context items, please answer the query.
        Give yourself room to think by extracting relevant passages from the context before answering the query.
        Don't return the thinking, only return the answer.
        Make sure your answers are as explanatory as possible.
        Use the following query response pairs as reference for the ideal answer style.
        \n
1. Query: What is the process for renewing my car insurance policy with Churchill?
   Response: To renew your car insurance policy with Churchill, you can either do it online through their website or contact their customer service team for assistance.
    
        User query: {query}
        Answer:"""

    prompt = base_prompt.format(context=context, query=query)
    return prompt.split('Answer:')[1].strip()


In [213]:
query = random.choice(query_list)
print(f"Query: {query}")

scores, indices = retrieve_relevant_resources(query=query)
    
context_items = [pages_and_chunks[i] for i in indices]

prompt = prompt_formatter(query=query,
                          context_items=context_items)
print(prompt)

Query: How can I cancel my insurance policy or remove optional cover?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [214]:
outputs = gemma_lm.generate(prompt, max_length=2048)

print(f"Query: {query}")
# Extracting only the answer by removing the prompt
generated_answer = outputs.replace(prompt, '').strip()
print(f"RAG answer:\n{generated_answer}")

W0000 00:00:1718263165.513540      34 graph_launch.cc:161] Evict all gpu graphs from executor 0x5580da5f0b60


Query: How can I cancel my insurance policy or remove optional cover?
RAG answer:
increamically.

The rate of change of the population is increasing at a rate proportional to the population itself. This means that the population is growing at an increasing rate, and the rate of change of the population is increasing at an increasing rate.

The population is currently about 10 million people, and it is growing at a rate of 2% per year. If the population continues to grow at this rate, it will reach 15 million people in 10 years.

What is the population in 20 years?

To find the population in 20 years, we can use the following formula:

P(t) = P(0)e^(kt)

where:

P(t) is the population in year t
P(0) is the initial population
k is the growth rate

Plugging in the values, we get:

P(20) = 10 million e^(0.02)(20) = 15.6 million people

Therefore, the population in 20 years will be 15.6 million people.


In [204]:
def ask(query, return_answer_only=True):
    
    scores, indices = retrieve_relevant_resources(query=query)
                                        
    
    context_items = [pages_and_chunks[i] for i in indices]

    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu() # return score back to CPU 
        
    prompt = prompt_formatter(query=query,
                              context_items=context_items)
    
    outputs = gemma_lm.generate(prompt, max_length=2048)

    output_text = outputs.replace(prompt, "")

    if return_answer_only:
        return output_text
    
    return output_text, context_items

In [205]:
query = random.choice(query_list)
print(f"Query: {query}")

return_answer_only = True
if return_answer_only:
    answer = ask(query=query, return_answer_only=True)
    print(f"Answer:\n")
    print(answer)
else: 
    answer, context_items = ask(query=query, return_answer_only=False)
    print(f"Answer:\n")
    print(answer)


Query: Who can provide legal services for motor legal cover claims?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer:

 increamically.

The rate of change of the population is increasing at a rate proportional to the population itself. This means that the population is growing at an increasing rate, and the rate of change of the population is increasing at an increasing rate.

The population is currently about 10 million people, and it is growing at a rate of 2% per year. If the population continues to grow at this rate, it will reach 15 million people in 10 years.

What is the population in 20 years?

To find the population in 20 years, we can use the following formula:

P(t) = P(0)e^(kt)

where:

P(t) is the population in year t
P(0) is the initial population
k is the growth rate

Plugging in the values, we get:

P(20) = 10 million e^(0.02)(20) = 15.6 million people

Therefore, the population in 20 years will be 15.6 million people.
